# Recommender Engine: Data Merging and Formatting

In [17]:
import os
import pandas as pd
import uuid
import string
import random

### Data Cleaning

In [2]:
## Import CSVs of each user and assign a unique user Id

#read the path to the user data
file_path = '/Users/angie/Documents/Recommender_Engine/User_Data'
#list all the files from the directory
file_list = pd.Series(os.listdir(file_path))
#make sure all the entries in the list are csvs
file_list=file_list[file_list.str.contains('csv')]

In [3]:
#read in and join all of the user data frames
data = pd.DataFrame()
for file in file_list:
            df_temp = pd.read_csv('/Users/angie/Documents/Recommender_Engine/User_Data/'+str(file))
            df_temp['user_id']=uuid.uuid4()  # create unique user id
            data = pd.concat([data,df_temp], ignore_index=True)
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()

,track_id,track_name,track_duration,track_popularity,track_uri,artist_name,artist_id,album_id,album_name,acousticness,...,celtic rock,world worship,christian indie,canadian contemporary country,anthem worship,vegas indie,australian country,black americana,christian rock,alberta country
0,4AYtqFyFbX0Xkc2wtcygTr,Soon You’ll Get Better (feat. The Chicks),201586,66,spotify:track:4AYtqFyFbX0Xkc2wtcygTr,Taylor Swift,06HL4z0CvFAxyc27GXpf02,1NAmidJlEaVgA3MpcPFYGq,Lover,0.90700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1JcIXOir94YUYBt2cXTzn2,Growing Sideways,255143,72,spotify:track:1JcIXOir94YUYBt2cXTzn2,Noah Kahan,2RQXRUsr4IW1f3mKyKsy4B,50ZenUP4O2Q5eCy2NRNvuz,Stick Season,0.74400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3UMrglJeju5yWyYIW6o99b,The Great War,240355,87,spotify:track:3UMrglJeju5yWyYIW6o99b,Taylor Swift,06HL4z0CvFAxyc27GXpf02,3lS1y25WAhcqJDATJK70Mq,Midnights (3am Edition),0.21900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3FFpGDzfkwJGjfIqM6BNMC,Love & War in Your Twenties,250547,66,spotify:track:3FFpGDzfkwJGjfIqM6BNMC,Jordy Searcy,0AV5z1x1RoOGeJWeJzziDz,0L9K2iqQGwfEhY96aHDiUB,Dark in the City,0.22000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7rC3P7tpWriaC4hYWKwGQd,Hey Lover!,162560,68,spotify:track:7rC3P7tpWriaC4hYWKwGQd,Wabie,3wOQaMNQU0rbzWyGXIudmE,076hhK2bCjHhQJqWY7wRh0,Cyclones and Hey Lover,0.00614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# fill in missing genres with 0
data.fillna(value=0,inplace=True)

#extract following features from large df
track_id=data.track_id
track_name=data.track_name.str.translate(str.maketrans('', '', string.punctuation)).str.lower()  #clean_track_names
artist_name=data.artist_name.str.lower() #clean_names
album_name=data.track_name.str.lower()

In [5]:
## list each songs genre as strings rather than one hot encoded

# create genredf, genres start at column index 36
genresdf=data.iloc[:,36:]

# if song, genre intersection ==1 append genre name to new genre column
genrecol=[]
for i in range(len(genresdf)):
    genres=[]
    for j in range(len(genresdf.columns)):
        if genresdf.iloc[i,j]==1:
            genres.append(genresdf.columns[j])
    genrecol.append(genres)

In [6]:
# Bin numerical features to produce descriptions
track_popularity_cat=pd.cut(data.track_popularity,bins=[-1,33,66,101],labels=['lowpopularity','midpopularity','highpopularity'])
acousticness_cat=pd.cut(data.acousticness,bins=[-1,.33,.66,1.01],labels=['lowacousticness','midacousticness','highacousticness'])
danceability_cat=pd.cut(data.danceability,bins=[-1,.33,.66,1.01],labels=['lowdanceability','middanceability','highdanceability'])
energy_cat=pd.cut(data.energy,bins=[-1,.33,.66,1.01],labels=['lowenergy','midenergy','highenergy'])
instrumentalness_cat=pd.cut(data.instrumentalness,bins=[-1,.33,.66,1.01],labels=['lowinstrumentalness','midinstrumentalness','highinstrumentalness'])
liveness_cat=pd.cut(data.liveness,bins=[-1,.33,.66,1.01],labels=['lowliveness','midliveness','highliveness'])
loudness_cat=pd.cut(data.loudness,bins=3,labels=['lowloudness','midloudness','highloudness'])
mode_cat=pd.cut(data['mode'],bins=[-1,.5,1.3],labels=['minormode','majormode'])
speechiness_cat=pd.cut(data.speechiness,bins=[-1,.33,.66,1.01],labels=['lowspeechiness','midspeechiness','highspeechiness'])
tempo_cat=pd.cut(data.tempo,bins=[-1,76,120,5000],labels=['slowtempo','midtempo','hightempo'])
valence_cat=pd.cut(data.valence,bins=[-1,.33,.66,1.01],labels=['lowvalence','midvalence','highvalence'])
albumpop_cat=pd.cut(data.album_popularity,bins=[-1,33,66,101],labels=['lowalbum_popularity','midalbum_popularity','highalbum_popularity'])
artist_popularity_cat=pd.cut(data.artist_popularity,bins=[-1,33,66,101],labels=['lowartist_popularity','midartist_popularity','highartist_popularity'])
year_cat=pd.cut(data.track_releaseYear,bins=[-1,1800,1950,1960,1970,1980,1990,2000,2012,2030],labels=['realrealold','realold','fifties','sixties','seventies','eighties','nineties','twothousands','presenttime'])

# if probability of song being in given key is greater than .5 then it is assigned to the key. songs can be in more than one key
key0_cat=pd.cut(data.key_0,bins=[-1,.5,1.1],labels=['','key0'])
key1_cat=pd.cut(data.key_1,bins=[-1,.5,1.1],labels=['','key1'])
key2_cat=pd.cut(data.key_2,bins=[-1,.5,1.1],labels=['','key2'])
key3_cat=pd.cut(data.key_3,bins=[-1,.5,1.1],labels=['','key3'])
key4_cat=pd.cut(data.key_4,bins=[-1,.5,1.1],labels=['','key4'])
key5_cat=pd.cut(data.key_5,bins=[-1,.5,1.1],labels=['','key5'])
key6_cat=pd.cut(data.key_6,bins=[-1,.5,1.1],labels=['','key6'])
key7_cat=pd.cut(data.key_7,bins=[-1,.5,1.1],labels=['','key7'])
key8_cat=pd.cut(data.key_8,bins=[-1,.5,1.1],labels=['','key8'])
key9_cat=pd.cut(data.key_9,bins=[-1,.5,1.1],labels=['','key9'])
key10_cat=pd.cut(data.key_10,bins=[-1,.5,1.1],labels=['','key10'])
key11_cat=pd.cut(data.key_11,bins=[-1,.5,1.1],labels=['','key11'])

In [7]:
#concatenating keys to make one variable similar to genres
key_df=pd.DataFrame({'key0':key0_cat,
                     'key1':key1_cat,
                     'key2':key2_cat,
                     'key3':key3_cat,
                     'key4':key4_cat,
                     'key5':key5_cat,
                     'key6':key6_cat,
                     'key7':key7_cat,
                     'key8':key8_cat,
                     'key9':key9_cat,
                     'key10':key10_cat,
                     'key11':key11_cat})

keycol=[]
for i in range(len(key_df)):
    keys=[]
    for j in range(len(key_df.columns)):
        if key_df.iloc[i,j]!='':
            keys.append(key_df.iloc[i,j])
    keycol.append(keys)

In [8]:
#features data frame
features=pd.DataFrame({'track_id':track_id,
          'track_name':track_name, 
          'artist_name':artist_name,
          'album_name':album_name,            
          'track_pop':track_popularity_cat,
          'acoustic':acousticness_cat,
          'danceability':danceability_cat,
          'energy':energy_cat,
          'instrumentalness':instrumentalness_cat,
          'liveness':liveness_cat,
          'loudness':loudness_cat,
          'mode':mode_cat,
          'speechiness':speechiness_cat,
          'tempo':tempo_cat,
          'valence':valence_cat,
          'albumpop':albumpop_cat,
          'artist_popularity':artist_popularity_cat,
          'year':year_cat,
          'genre':genrecol,
          'key':keycol
        })

In [9]:
#punctuation removal
features['genre']=features.genre.astype('str').str.translate(str.maketrans('', '', string.punctuation))
features['key']=features.key.astype('str').str.translate(str.maketrans('', '', string.punctuation))
features=features.astype('str')
features.head()

,track_id,track_name,artist_name,album_name,track_pop,acoustic,danceability,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence,albumpop,artist_popularity,year,genre,key
0,4AYtqFyFbX0Xkc2wtcygTr,soon you’ll get better feat the chicks,taylor swift,soon you’ll get better (feat. the chicks),midpopularity,highacousticness,middanceability,lowenergy,lowinstrumentalness,lowliveness,highloudness,majormode,lowspeechiness,hightempo,midvalence,highalbum_popularity,highartist_popularity,presenttime,pop,key0
1,1JcIXOir94YUYBt2cXTzn2,growing sideways,noah kahan,growing sideways,highpopularity,highacousticness,highdanceability,lowenergy,lowinstrumentalness,lowliveness,highloudness,majormode,lowspeechiness,midtempo,midvalence,highalbum_popularity,highartist_popularity,presenttime,pop,key7
2,3UMrglJeju5yWyYIW6o99b,the great war,taylor swift,the great war,highpopularity,lowacousticness,middanceability,highenergy,lowinstrumentalness,lowliveness,highloudness,majormode,lowspeechiness,midtempo,midvalence,highalbum_popularity,highartist_popularity,presenttime,pop,key5
3,3FFpGDzfkwJGjfIqM6BNMC,love war in your twenties,jordy searcy,love & war in your twenties,midpopularity,lowacousticness,highdanceability,midenergy,lowinstrumentalness,lowliveness,highloudness,majormode,lowspeechiness,midtempo,midvalence,midalbum_popularity,midartist_popularity,presenttime,stomp and holler indiecoustica,key2
4,7rC3P7tpWriaC4hYWKwGQd,hey lover,wabie,hey lover!,highpopularity,lowacousticness,middanceability,highenergy,lowinstrumentalness,midliveness,highloudness,majormode,lowspeechiness,midtempo,midvalence,midalbum_popularity,midartist_popularity,presenttime,bedroom pop,key4


### LDA Data Formatting

In [10]:
# create description for each song containing all features
description=pd.DataFrame(index=range(len(features)),columns=range(1))
description.columns=['attribute']
for i in range(len(features)):
    attribute=''
    for j in range(1,len(features.columns)):
        attribute=attribute+features.iloc[i,j]+' '
    description.attribute[i]=attribute
    
#Create label
description['label']=list(features.track_id)       


In [11]:
#keep label and attributes only
songs=description.loc[:,['label','attribute']]
#rename columns
songs.columns=['track_name','_description']
songs.head()

,track_name,_description
0,4AYtqFyFbX0Xkc2wtcygTr,soon you’ll get better feat the chicks taylor ...
1,1JcIXOir94YUYBt2cXTzn2,growing sideways noah kahan growing sideways h...
2,3UMrglJeju5yWyYIW6o99b,the great war taylor swift the great war highp...
3,3FFpGDzfkwJGjfIqM6BNMC,love war in your twenties jordy searcy love &...
4,7rC3P7tpWriaC4hYWKwGQd,hey lover wabie hey lover! highpopularity lowa...


### Matrix Factorization Data Formatting

In [12]:
#keep user_id, track_id,and track_name and artist_name for recommendations
ratings=data[['user_id','track_id','track_name','artist_name']].reset_index(drop=True)

#add 1 to indicate user has song listed in song library
ratings['rating']=1
ratings.head()

,user_id,track_id,track_name,artist_name,rating
0,58d196bb-463e-403c-84e5-bf7bb531fdfe,4AYtqFyFbX0Xkc2wtcygTr,Soon You’ll Get Better (feat. The Chicks),Taylor Swift,1
1,58d196bb-463e-403c-84e5-bf7bb531fdfe,1JcIXOir94YUYBt2cXTzn2,Growing Sideways,Noah Kahan,1
2,58d196bb-463e-403c-84e5-bf7bb531fdfe,3UMrglJeju5yWyYIW6o99b,The Great War,Taylor Swift,1
3,58d196bb-463e-403c-84e5-bf7bb531fdfe,3FFpGDzfkwJGjfIqM6BNMC,Love & War in Your Twenties,Jordy Searcy,1
4,58d196bb-463e-403c-84e5-bf7bb531fdfe,7rC3P7tpWriaC4hYWKwGQd,Hey Lover!,Wabie,1


In [13]:
# compare songs in the song(LDA) df and ratings (Matrix Factorization) df
titles_songs = set(songs.track_name.to_list())
titles_ratings = set(ratings.track_id.to_list())

# songs with both rating and description
intersection = titles_songs.intersection(titles_ratings)
len(intersection)

8889

In [14]:
# some songs are duplicates
print(f"Found {songs[songs.track_name.duplicated()].shape[0]} duplicate entries in the songs dataset.")
intersection = intersection.difference(set(songs[songs.track_name.duplicated()].track_name.to_list()))
songs = songs[songs.track_name.isin(intersection)]
print(f"Found {songs[songs.track_name.duplicated()].shape[0]} duplicate entries in the songs dataset.")


# the ratings dataset does not have duplicate item problems, so we just drop the titles we dropped in the songs dataset.
ratings = ratings[ratings.track_id.isin(intersection)]

# check to see that both datasets contain the same songs.
try:
    assert len(set(ratings.track_id.to_list())) == songs.shape[0]
    print("Dataset song entries match.")
except AssertionError:
    raise Exception("Dataset song entries differ.")

Found 513 duplicate entries in the songs dataset.
Found 0 duplicate entries in the songs dataset.
Dataset song entries match.


In [15]:
# pivot the table
R = pd.pivot_table(data=ratings, values = ["rating"], index=["user_id"], columns=["track_id"])

# remove the level on top of song names called "Value"
R.columns = R.columns.droplevel()

# remove leftover columns name from pivot operation
R.columns.name = ""

# fill in the NaNs with 0's
R.fillna(0, inplace=True)

In [18]:
# 10 songs to hold out from the ratings matrix - we will do out-of-matrix prediciton on them

random.seed(200)
heldout_idx = random.choices(range(len(ratings)),k=10)
heldout_id = R.columns[heldout_idx]
heldout_songs=ratings.loc[[x in heldout_id for x in ratings.track_id],['track_name','artist_name']].reset_index(drop=True)
heldout_ratings = R[heldout_id]
R.drop(heldout_id, axis=1, inplace=True)

print(f"dropped '{heldout_songs}' from the ratings matrix")

dropped '                                          track_name               artist_name
0                                   Love You Goodbye             One Direction
1                                      Wherever I Go               OneRepublic
2                                      The Uber Song                      DRAM
3                                         Unfaithful                   Rihanna
4                                       Made to Love               John Legend
5                                   Nobody's Perfect                   J. Cole
6                                         Free Smoke                     Drake
7                                    Redemption Song  Bob Marley & The Wailers
8                     Big Girls Don't Cry (Personal)                    Fergie
9  Défiler - Bande originale de la capsule No. 5 ...                   Stromae' from the ratings matrix


In [23]:
# Rating Matrix with track id as identifier and held out songs removed
R

,005lwxGU1tms6HGELIcUv9,00CqEmnPLFKDhAb3cuu6Cs,00L9IGHexZLnscZiQpUWTG,00Mb3DuaIH1kjrwOku9CGU,00S66PHVusji3COAuNUpLP,00WubJUWuC6V8PKvqaHmQ2,00bd2zIw68UZUvtVdR9Zh3,00kbBKPcMWN7636XbdhhWu,00s2jnZ4cVZssim4VqrszP,00uaBUnmyGxBUSwG37sWHx,...,7zWwTeVvMM6juC1Kru1pu3,7za1TCWFFPF9ZSUZutkds5,7zbX1txfcs42K2PlhFJNNE,7ze7yEV1xvxPxuPaF0oUUU,7zfmhBiUvGoHz76NO1d5bZ,7zhzzFiASS8lOHLrooBI1t,7znoroIV2dSMcjpv7ZxnLh,7zqM7Wm2xJLZ50qAnTLoHD,7zsXy7vlHdItvUSH8EwQss,7zy1cb4QkODyjddpmTw7Nm
user_id,,,,,,,,,,,,,,,,,,,,,
27ff0962-54e3-4661-b3e5-4a44d3748375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
357f28a4-c9e8-4092-a4f8-4dc63a23d8af,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48b72bf1-cbf1-4352-888b-4236f1f3f2eb,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4c5debc6-e771-4051-b14a-24a744a98cb5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4d40d3e7-6c45-4f1d-83ce-04ab0737499e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58d196bb-463e-403c-84e5-bf7bb531fdfe,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6d527e87-f848-4120-a92c-33b0d47ca994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76c2594f-08e2-4371-ad2c-dc2f910694ca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c709fb1c-a12a-41eb-ae8d-517cda03e0ae,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
# assert that the pivot creation was successful
assert R.loc[R.index[5], '005lwxGU1tms6HGELIcUv9']==ratings[(ratings.user_id == R.index[5]) & (ratings.track_id=='005lwxGU1tms6HGELIcUv9')].rating.values

print(f"Ratings matrix ready with {R.shape[0]} users and {R.shape[1]} songs.")
r = R.values
sparsity = float(len(r.nonzero()[0]))
sparsity /= (r.shape[0] * r.shape[1])
sparsity *= 100
print(f"Matrix sparsity: {round(sparsity, 2)}%")

Ratings matrix ready with 11 users and 8405 songs.
Matrix sparsity: 9.09%


### Export CSVs

In [25]:
R.to_csv('ratings_matrix.csv')
songs.to_csv('LDA_data.csv')